<a href="https://colab.research.google.com/github/DanieleBaiocco/NLPProject/blob/dev_2/nlpproject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
from google.colab import drive
import pandas as pd
import os
from sklearn.preprocessing import LabelBinarizer
import numpy as np
from collections import defaultdict
from transformers import BertTokenizer, BertModel, BertForSequenceClassification
import transformers
from pathlib import Path
import pickle
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, Dataset
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import Optimizer
from tqdm import tqdm
import torch.optim as optim
from sklearn.metrics import f1_score
import matplotlib.pyplot as plt
import copy
from abc import ABC, abstractmethod

In [5]:
mount_directory = "/content/drive"
dataset_path = os.path.join(mount_directory, 'MyDrive/MELD/MELD_train_efr.json')
model_card = 'bert-base-uncased'
drive.mount(mount_directory)

Mounted at /content/drive


In [6]:
def fit_onehotencoder(data: pd.core.series.Series) -> LabelBinarizer:
  onehotencoder = LabelBinarizer()
  data_flattened = np.concatenate(data.values)
  onehotencoder.fit(data_flattened)
  return onehotencoder

In [7]:
#sbagliato, deve esserci un tokentypeids per ogni token e deve essere riferito a uno speaker particolare
def map_to_order_of_occurrence(data: list) -> list:
    # When a new entry is added, its value is computed using the lambda function
    dict_order_of_occurrence = defaultdict(lambda: len(dict_order_of_occurrence)+1)
    order_of_occurrence = [dict_order_of_occurrence[element] for element in data]
    return order_of_occurrence

In [8]:
def replace_none_with_zero(data: list) -> list:
    return [0 if x is None else x for x in data]

In [9]:
def tokenize_dialogue(dialogue_text: list, tokenizer,  speakers = None) -> list:
    if speakers!=None:
      tokenized_dialogue = [[speakers[idx]] + tokenizer.tokenize(utterance) + [tokenizer.sep_token] for idx, utterance in enumerate(dialogue_text)]
    else:
      tokenized_dialogue = [tokenizer.tokenize(utterance) + [tokenizer.sep_token] for utterance in dialogue_text]
    tokenized_dialogue.insert(0, [tokenizer.cls_token])
    flattened_tokens = [token for sublist in tokenized_dialogue for token in sublist]
    return flattened_tokens

In [10]:
def add_padding(x: list, max_x_length: int, pad_value, pad_length = None) -> list:
    x_length = len(x)
    num_pad_values = max_x_length - x_length
    if pad_length == None:
      padded_x = x + [pad_value] * num_pad_values
    else:
      pad_list = [pad_value] * pad_length
      pad_lists = np.tile(pad_list, (num_pad_values, 1))
      padded_x = np.concatenate((x, pad_lists), axis=0)
    return padded_x

In [11]:
def build_alternative_dialogue(utterances: pd.core.series.Series, speakers: pd.core.series.Series) -> pd.core.series.Series:
  dialogue_column = []
  for dialogue_index, dialogue in utterances.items():
    dialogue_column.append(tokenize_dialogue(dialogue, tokenizer, speakers[dialogue_index]))
  return pd.Series(dialogue_column)

In [12]:
tokenizer = BertTokenizer.from_pretrained(model_card, do_lower_case=True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [13]:
df = pd.read_json(dataset_path)
column_names = ['emotions', 'speakers', 'triggers', 'dialogues', 'dialogues_ids', 'attention_masks']
df_standard = pd.DataFrame(columns = column_names)
emotions_onehotencoder = fit_onehotencoder(df['emotions'])
df_standard['speakers'] = df['speakers'].apply(lambda x: map_to_order_of_occurrence(x))
df_standard['emotions'] = df['emotions'].apply(lambda x: emotions_onehotencoder.transform(x))
df_standard['triggers'] = df['triggers'].apply(lambda x: replace_none_with_zero(x))
df_variation = copy.deepcopy(df_standard)
df_standard['dialogues'] = df['utterances'].apply(lambda x: tokenize_dialogue(x, tokenizer))
df_variation['dialogues'] = build_alternative_dialogue(df['utterances'], df_standard['speakers'])

In [14]:
num_emotions =  len(emotions_onehotencoder.classes_)
max_num_utterances = df['utterances'].apply(len).max()

In [15]:
def save_dataframe(df):
    _folder = Path.cwd().joinpath("dataframes")
    if not _folder.exists():
        _folder.mkdir(parents=True)

    df_path = Path.joinpath(_folder, 'df_MELD_efr'+'.pkl')
    df.to_pickle(df_path)

In [16]:
def load_dataframe():
    df_path = 'dataframes/df_MELD_efr.pkl'
    if not os.path.exists(df_path):
        raise FileNotFoundError("{0} dataframe does not exist!".format(df_path))

    with open(df_path, 'rb') as file:
        df = pickle.load(file)
    return df

In [17]:
def split_dataframe(orginal_df: pd.DataFrame, seed: int):
    train, test_validation = train_test_split(orginal_df, test_size=0.2, random_state=seed)
    validation, test = train_test_split(test_validation, test_size=0.5, random_state=seed)
    return train.reset_index(drop=True), validation.reset_index(drop=True), test.reset_index(drop=True)

In [18]:
class CustomDataset(Dataset):
    def __init__(self, speakers: pd.core.series.Series,
                 dialogues:  pd.core.series.Series,
                 emotions:  pd.core.series.Series,
                 triggers:  pd.core.series.Series,
                 device,
                 pad_token: str,
                 max_num_utterances):
        self.max_dialogue_length = dialogues.apply(len).max()
        self.max_num_utterances = max_num_utterances
        self.dialogues = dialogues.apply(lambda x: add_padding(x, self.max_dialogue_length, pad_token))
        self.dialogues_ids = self.dialogues.apply(lambda x: tokenizer.convert_tokens_to_ids(x))
        self.attention_masks = self.dialogues.apply(lambda x: [1 if token != pad_token else 0 for token in x])
        self.speakers = speakers.apply(lambda x: add_padding(x, self.max_num_utterances, pad_value=0))
        num_emotion_classes = len(emotions[0][0])
        self.emotions = emotions.apply(lambda x: add_padding(x, self.max_num_utterances, pad_value = 0, pad_length = num_emotion_classes))
        self.triggers = triggers.apply(lambda x: add_padding(x, self.max_num_utterances, pad_value=0))
        self.device = device

    def __len__(self):
        return len(self.dialogues)

    def __getitem__(self, idx):
        speakers = torch.tensor(self.speakers.iloc[idx], dtype=torch.long).to(device)
        dialogues_ids = torch.tensor(self.dialogues_ids.iloc[idx], dtype=torch.long).to(device)
        dialogues_masks =  torch.tensor(self.attention_masks.iloc[idx], dtype=torch.long).to(device)
        emotions = torch.tensor(self.emotions.iloc[idx], dtype=torch.float32).to(device)
        triggers = torch.tensor(self.triggers.iloc[idx], dtype=torch.float32).to(device)
        return  speakers, dialogues_ids, dialogues_masks, emotions, triggers

In [19]:
def create_dataloader(df: pd.core.frame.DataFrame, device, tokenizer, max_num_utterances, batch_size) -> torch.utils.data.dataloader.DataLoader :
    dataset =  CustomDataset(speakers = df['speakers'],
                                dialogues = df['dialogues'],
                                emotions = df['emotions'],
                                triggers = df['triggers'],
                                device = device,
                                pad_token = tokenizer.pad_token,
                                max_num_utterances = max_num_utterances)
    return DataLoader(dataset, batch_size = batch_size, shuffle = True)

In [20]:
class AbstractBERTModel(ABC, torch.nn.Module):
    def __init__(self, model_card, tokenizer, num_emotions, max_num_utterances, freeze_embedding_layer=False):
        super(AbstractBERTModel, self).__init__()

        self.tokenizer = tokenizer
        self.max_num_utterances = max_num_utterances

        # Load pre-trained BERT model and tokenizer
        self.bert_model = BertModel.from_pretrained(model_card).to(device)

        if freeze_embedding_layer:
            for param in self.bert_model.embeddings.parameters():
                param.requires_grad = False

        self.representation_length = self.bert_model.config.hidden_size

        self.emotion_classifier = nn.Sequential(
            nn.Linear(self.representation_length, 512),
            nn.ReLU(),
            nn.Linear(512, num_emotions),
            nn.Softmax(dim=2)
        )

        self.trigger_prediction = self._create_trigger_module()

    @abstractmethod
    def _create_trigger_module(self):
        pass

    @abstractmethod
    def forward(self, token_type_ids, input_ids, attention_mask):
        pass

    def bert_forward(self, input_ids, attention_mask):
        bert_output_1 = self.bert_model(input_ids=input_ids, attention_mask=attention_mask).last_hidden_state
        # Extract [SEP] token representations
        sep_indices = (input_ids == self.tokenizer.sep_token_id).nonzero()
        batch_size = input_ids.shape[0]
        sep_representations = torch.zeros((batch_size, self.max_num_utterances, self.representation_length)).to(device)
        dialogue_masks = torch.zeros((batch_size, self.max_num_utterances))
        for idx in range(batch_size):
            sep_indices_idx = sep_indices[sep_indices[:,0] == idx][:,1]
            sep_indices_idx_range = range(len(sep_indices_idx))
            dialogue_masks[idx, sep_indices_idx_range] = 1
            sep_representations[idx, sep_indices_idx_range, :] = bert_output_1[idx, sep_indices_idx, :]
        return sep_representations, dialogue_masks

In [21]:
class CustomBERTModel(AbstractBERTModel):
    def __init__(self, model_card, tokenizer, num_emotions, max_num_utterances, gru_hidden_size, freeze_embedding_layer=False):
        super(CustomBERTModel, self).__init__(model_card, tokenizer, num_emotions, max_num_utterances, freeze_embedding_layer)
        self.linear_trigger = nn.Linear(gru_hidden_size * 2, 1)


    def _create_trigger_module(self):
        return nn.GRU(
            input_size=self.representation_length + num_emotions + 1,
            hidden_size=gru_hidden_size,
            batch_first=True,
            bidirectional=True
        )

    def forward(self, token_type_ids, input_ids, attention_mask):
        sep_representations, dialogue_masks = self.bert_forward(input_ids, attention_mask)
        emotion_predictions = self.emotion_classifier(sep_representations) * dialogue_masks.unsqueeze(-1)
        concatenated_input = torch.cat([sep_representations, emotion_predictions, token_type_ids.unsqueeze(-1)], dim=-1)
        trigger_output, _ = self.trigger_prediction(concatenated_input)
        trigger_output_single_value = self.linear_trigger(trigger_output).squeeze(-1)
        trigger_output_sigmoid = torch.sigmoid(trigger_output_single_value) * dialogue_masks
        return emotion_predictions, trigger_output_sigmoid, dialogue_masks

In [22]:
class BaselineBERTModel(AbstractBERTModel):
    def __init__(self, model_card, tokenizer, num_emotions, max_num_utterances, freeze_embedding_layer=False):
        super(BaselineBERTModel, self).__init__(model_card, tokenizer, num_emotions, max_num_utterances, freeze_embedding_layer)

    def _create_trigger_module(self):
        return nn.Sequential(
            nn.Linear(self.representation_length, 512),
            nn.ReLU(),
            nn.Linear(512, 1),
            nn.Sigmoid()
        )

    def forward(self, _, input_ids, attention_mask):
        sep_representations, dialogue_masks = self.bert_forward(input_ids, attention_mask)
        emotion_predictions = self.emotion_classifier(sep_representations) * dialogue_masks.unsqueeze(-1)
        trigger_output = self.trigger_prediction(sep_representations) * dialogue_masks.unsqueeze(-1)
        trigger_output = trigger_output.squeeze(dim=-1)
        return emotion_predictions, trigger_output, dialogue_masks

In [30]:
class RandomClassifier(torch.nn.Module):
    def __init__(self, seed):
        super(RandomClassifier, self).__init__()
        self.seed = seed

    def _build_dialogue_masks(self, data):
        num_emotions = len(data[0][0])
        for dialogue in data:



    def forward(self, data):
         dialogue_masks = self._build_dialogue_masks(data)

         return 0


## ss

In [24]:
class CustomTraining():
    def __init__(self, training_loader: torch.utils.data.dataloader.DataLoader,
                 validation_loader: torch.utils.data.dataloader.DataLoader,
                 test_loader: torch.utils.data.dataloader.DataLoader,
                 emotions_onehotencoder,
                 device: str,
                 epochs=20,
                 seed=42):
        self.training_loader = training_loader
        self.validation_loader = validation_loader
        self.test_loader = test_loader
        self.emotions_onehotencoder = emotions_onehotencoder
        self.epochs = epochs
        self.device = device
        self.seed = seed

    def _forward_step(_, speakers, dialogues_ids, dialogues_masks, model):
            emotion_pred, trigger_pred, dialogue_mask = model(speakers, dialogues_ids, dialogues_masks)
            return emotion_pred, trigger_pred, dialogue_mask

    def _compute_loss(_, emotion_pred, emotions, trigger_pred, triggers):
            emotion_loss = F.binary_cross_entropy_with_logits(emotion_pred, emotions)
            trigger_loss = F.binary_cross_entropy_with_logits(trigger_pred, triggers)
            loss = emotion_loss + trigger_loss
            return loss

    def _remove_pad(_, batch_idx, target, predictions, dialogue_length):
            target_nopad = target[batch_idx, :dialogue_length]
            predictions_nopad = predictions[batch_idx, :dialogue_length]
            return target_nopad, predictions_nopad


    def _remove_pad_predictions(self, batch_idx, emotions, emotion_predictions, triggers, trigger_predictions, dialogue_mask):
            dialogue_length = (dialogue_mask[batch_idx] == 0).nonzero()[0][0].item()
            emotions_nopad, emotion_pred_nopad = self._remove_pad(batch_idx, emotions, emotion_predictions, dialogue_length)
            triggers_nopad, trigger_pred_nopad = self._remove_pad(batch_idx, triggers, trigger_predictions, dialogue_length)
            return emotions_nopad, emotion_pred_nopad, triggers_nopad, trigger_pred_nopad

    def _update_metric_arrays(_, emotions_nopad, emotion_pred_nopad, triggers_nopad, triggers_pred_nopad,
                        emotions_flat, emotions_pred_flat, triggers_flat, triggers_pred_flat,
                        f1_seq_emotions, f1_seq_triggers):
            emotions_flat.extend(emotions_nopad.tolist())
            emotions_pred_flat.extend(emotion_pred_nopad.tolist())
            triggers_flat.extend(triggers_nopad.tolist())
            triggers_pred_flat.extend(triggers_pred_nopad.tolist())
            f1_seq_emotions.append(f1_score(emotions_nopad, emotion_pred_nopad, average = 'micro'))
            f1_seq_triggers.append(f1_score(triggers_nopad, triggers_pred_nopad, average = 'micro'))


    def _get_metric_results(_, flattened_emotions, flattened_emotions_pred, flattened_triggers, flattened_triggers_pred, f1_sequence_emotions, f1_sequence_triggers):
                  #emotion_pred_labels = torch.eye(emotion_pred.size(2))[torch.argmax(emotion_pred, dim=-1)]
            avg_f1_sequence_emotion = sum(f1_sequence_emotions) / len(f1_sequence_emotions)
            avg_f1_sequence_trigger = sum(f1_sequence_triggers) / len(f1_sequence_triggers)
            f1_flattened_emotion = f1_score(flattened_emotions, flattened_emotions_pred, average='micro')
            f1_flattened_trigger = f1_score(flattened_triggers, flattened_triggers_pred, average='micro')
            return avg_f1_sequence_emotion, avg_f1_sequence_trigger, f1_flattened_emotion, f1_flattened_trigger

    def _compute_metrics(self, emotions, emotion_pred, triggers, trigger_pred, dialogue_mask,
                 emotions_flat, emotions_pred_flat, triggers_flat, triggers_pred_flat,
                f1_sequence_emotions, f1_sequence_triggers):
            emotions = torch.argmax(emotions, dim=-1)
            emotion_pred = torch.argmax(emotion_pred, dim=-1)
            trigger_pred = (trigger_pred > 0.5).float()
            for batch_idx in range(emotion_pred.size(0)):
                emotions_nopad, emotion_pred_nopad, triggers_nopad, triggers_pred_nopad = self._remove_pad_predictions(batch_idx,
                                                                                                                       emotions,
                                                                                                                       emotion_pred,
                                                                                                                       triggers,
                                                                                                                       trigger_pred,
                                                                                                                       dialogue_mask)
                if 1.0 in triggers_pred_nopad:
                  print(triggers_pred_nopad)
                self._update_metric_arrays(emotions_nopad, emotion_pred_nopad, triggers_nopad, triggers_pred_nopad,
                        emotions_flat, emotions_pred_flat, triggers_flat, triggers_pred_flat,
                        f1_sequence_emotions, f1_sequence_triggers)

    def _set_loop_info(self,  loop,  set_name, loss, avg_loss, avg_f1_sequence_emotion, f1_flattened_emotion, avg_f1_sequence_trigger, f1_flattened_trigger):
            loop.set_description(f'{set_name} set.')
            loop.set_postfix({'loss': f'{loss.item():.5}', 'loss_average': f'{avg_loss:.5}',
                              'f1_sequence_emotion': f'{avg_f1_sequence_emotion:.5}',
                              'f1_flattened_emotion': f'{f1_flattened_emotion:.5}',
                              'f1_sequence_trigger': f'{avg_f1_sequence_trigger:.5}',
                              'f1_flattened_trigger': f'{f1_flattened_trigger:.5}',})

    def train_step(self, model: nn.Module, optimizer: Optimizer):
        total_loss = 0
        train_step = 0
        emotions_flat, emotions_pred_flat ,triggers_flat, triggers_pred_flat, f1_sequence_emotions, f1_sequence_triggers = [],[],[],[],[],[]
        loop = tqdm(enumerate(self.training_loader, 0), total=len(self.training_loader))
        for _,data in loop:
            optimizer.zero_grad()
            train_step += 1
            speakers, dialogues_ids, dialogues_masks, emotions, triggers = data
            emotion_pred, trigger_pred, dialogue_mask = self._forward_step(speakers, dialogues_ids, dialogues_masks, model)
            loss = self._compute_loss(emotion_pred, emotions, trigger_pred, triggers)
            loss.backward()
            optimizer.step()
            self._compute_metrics(emotions, emotion_pred, triggers, trigger_pred, dialogue_mask,
                 emotions_flat, emotions_pred_flat ,triggers_flat, triggers_pred_flat, f1_sequence_emotions, f1_sequence_triggers)

            avg_f1_sequence_emotion, avg_f1_sequence_trigger, f1_flattened_emotion, f1_flattened_trigger = self._get_metric_results(emotions_flat, emotions_pred_flat ,triggers_flat, triggers_pred_flat,
                                                                                                                                 f1_sequence_emotions,
                                                                                                                                 f1_sequence_triggers)
            total_loss += loss.item()
            avg_loss = total_loss / train_step
            self._set_loop_info(loop, 'Train',  loss, avg_loss, avg_f1_sequence_emotion, f1_flattened_emotion, avg_f1_sequence_trigger, f1_flattened_trigger)
        return avg_loss, avg_f1_sequence_emotion, f1_flattened_emotion, avg_f1_sequence_trigger, f1_flattened_trigger

    def _populate_history(_, history, loss, f1seq_emotion, f1flat_emotion, f1seq_trigger, f1flat_trigger):
        history['loss'].append(loss)
        history['f1seq_emotion'].append(f1seq_emotion)
        history['f1flat_emotion'].append(f1flat_emotion)
        history['f1seq_trigger'].append(f1seq_trigger)
        history['f1flat_trigger'].append(f1flat_trigger)

    def train(self, model, optimizer, patience):
            train_history = {'loss': [], 'f1seq_emotion': [], 'f1flat_emotion': [],
                            'f1seq_trigger': [], 'f1flat_trigger': []}

            val_history = {'loss': [], 'f1seq_emotion': [], 'f1flat_emotion': [],
                          'f1seq_trigger': [], 'f1flat_trigger': []}
            best_val_loss = float('inf')
            for epoch in range(self.epochs):
                model.train()
                train_loss, f1seq_emotion_train, f1flat_emotion_train, f1seq_trigger_train, f1flat_trigger_train = self.train_step(model, optimizer)
                self._populate_history(train_history, train_loss, f1seq_emotion_train, f1flat_emotion_train, f1seq_trigger_train, f1flat_trigger_train)
                model.eval()
                val_loss, f1seq_emotion_val, f1flat_emotion_val, f1seq_trigger_val, f1flat_trigger_val = self.evaluate(model, self.validation_loader, split_name = 'Validation')
                self._populate_history(val_history, val_loss, f1seq_emotion_val, f1flat_emotion_val, f1seq_trigger_val, f1flat_trigger_val)
                if val_loss < best_val_loss:
                    best_val_loss = val_loss
                    early_stopping_counter = 0
                else:
                    early_stopping_counter += 1
                # Check if training should stop
                if early_stopping_counter >= patience:
                    print(f'Early stopping at epoch {epoch}...')
                    break
            return train_history, val_history

    def test(self):
              _, f1seq_emotion_test, f1flat_emotion_test, f1seq_trigger_test, f1flat_trigger_test = self.evaluate(model, self.test_loader, split_name = 'Test')

    def evaluate(self, dataloader, model,  is_test=False):
              total_loss = 0
              evaluate_step = 0
              flattened_emotions, flattened_emotions_pred,flattened_triggers, flattened_triggers_pred, f1_sequence_emotions, f1_sequence_triggers = [],[],[],[],[],[]
              with torch.no_grad():
                  loop = tqdm(enumerate(dataloader, 0), total=len(dataloader))
                  for _, data in loop:
                      evaluate_step += 1
                      speakers, dialogues_ids, dialogues_masks, emotions, triggers = data
                      emotion_pred, trigger_pred, dialogue_mask = self._forward_step(speakers, dialogues_ids, dialogues_masks, model)
                      loss = self._compute_loss(emotion_pred, emotions, trigger_pred, triggers)
                      self._compute_metrics(emotions, emotion_pred, triggers, trigger_pred, flattened_emotions, flattened_emotions_pred, flattened_triggers, flattened_triggers_pred, f1_sequence_emotions, f1_sequence_triggers)
                      total_loss += loss.item()
                  av_loss = 0.0
                  if not is_test:
                    avg_loss = total_loss / evaluate_step
                  avg_f1_sequence_emotion, avg_f1_sequence_trigger, f1_flattened_emotion, f1_flattened_trigger = self._get_metric_results(flattened_emotions,
                                                                                                                                 flattened_emotions_pred,
                                                                                                                                 flattened_triggers,
                                                                                                                                 flattened_triggers_pred,
                                                                                                                                 f1_sequence_emotions,
                                                                                                                                 f1_sequence_triggers)
                  if is_test:
                    split_name = 'Test'
                  else:
                    split_name='Validation'
                  self._set_loop_info(loop, split_name,  loss, avg_loss, avg_f1_sequence_emotion, f1_flattened_emotion, avg_f1_sequence_trigger, f1_flattened_trigger)
              return avg_loss, avg_f1_sequence_emotion, f1_flattened_emotion, avg_f1_sequence_trigger, f1_flattened_trigger

## ssd

In [25]:
def build_trainer(dataframe: pd.core.frame.DataFrame,
          device: str,
          tokenizer: transformers.models.bert.tokenization_bert.BertTokenizer,
          max_num_utterances: int,
          batch_size: int):
    df_train, df_val, df_test = split_dataframe(dataframe, 42)
    dataloader_train = create_dataloader(df_train, device, tokenizer, max_num_utterances, batch_size)
    dataloader_val = create_dataloader(df_val, device, tokenizer, max_num_utterances, batch_size)
    dataloader_test = create_dataloader(df_test, device, tokenizer, max_num_utterances, batch_size)
    trainer = CustomTraining(dataloader_train, dataloader_val, dataloader_test, emotions_onehotencoder, device)
    return trainer

In [31]:
random_classifier = RandomClassifier( seed = 42)
random_classifier(df_standard['emotions'])

7


0

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
batch_size = 8
trainer  = build_trainer(df_standard, device, tokenizer, max_num_utterances, batch_size)

In [ ]:
optimizer = optim.Adam(model.parameters(), lr=0.001)
train_history, val_history = trainer.train(model, optimizer, patience= 5)

In [ ]:
def plot_metrics(train_history, val_history, metric_names, metric_labels):
    epochs = range(1, len(train_history['loss']) + 1)

    plt.figure(figsize=(10, 6))

    for train_metric, val_metric, label in zip(metric_names[::2], metric_names[1::2], metric_labels):
        plt.plot(epochs, train_history[train_metric], label=f'Training {label}', linestyle='-', linewidth=2)
        plt.plot(epochs, val_history[val_metric], label=f'Validation {label}', linestyle='--', linewidth=2)

    plt.title('Training and Validation Metrics')
    plt.xlabel('Epochs')
    plt.ylabel('Metric Value')
    plt.legend()
    plt.show()

In [ ]:
trigger_metric_names = ['f1seq_trigger_train', 'f1flat_trigger_train', 'f1seq_trigger_val', 'f1flat_trigger_val']
trigger_metric_labels = ['F1 Score - Seq Trigger', 'F1 Score - Flat Trigger']

plot_metrics(train_history, val_history, trigger_metric_names, trigger_metric_labels)

In [ ]:
plt.plot(epochs, train_history['loss'], label='Training Loss', linestyle='-', linewidth=2)
plt.plot(epochs, val_history['loss'], label='Validation Loss', linestyle='--', linewidth=2)